In [ ]:
%matplotlib inline

import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import glob
from natsort import natsorted

from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
import math, getopt, sys
import datetime
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from skimage.io import imshow
from scipy import ndimage as ndi

from skimage import exposure, measure
from skimage.feature import canny
from skimage.filters import sobel, rank, gaussian, median, threshold_adaptive
from skimage.morphology import disk, watershed, square

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import ADASYN
from sklearn.metrics import hamming_loss

from skimage import morphology

from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

In [ ]:
clip_limit = 0.01
bins = 15

sigma_for_images = 1

use_gaussian = 1
use_median = 0

use_equalize_hist = 0
use_adapthist = 1
use_rescale_intensity = 0

save_images = 0
save_histograms = 0

use_pca_for_segments = 1
use_pca_for_learning = 0
use_pca_before_learning = 1

n_components_for_2D_segments = 5
n_components_for_3D_segments = 1
n_components_before_learning = 20

use_3D = 0

x_parts, y_parts, z_parts = 5, 5, 5

cv_inner_folds = 10
cv_outer_folds = 2

#### Function to generate n_arr (almost) equal-sized arrays from arr

In [ ]:
def chunks(arr, n_arr):
    n_elems = int(math.ceil(len(arr)/n_arr))
    return [arr[i:i + n_elems] for i in range(0, len(arr), n_elems)]

#### Function to load images from file being in defined path, deleting all zero vectors from image

In [ ]:
def load_image(path, file):
    
    filepath = os.path.join(path, file)
    img = nib.load(filepath)
    img_data = img.get_data()
    img_data = img_data[:,:,:,0]
    
    return img_data

In [ ]:
def cut_2D_image(img_data):
    
    img_data = np.delete(img_data, x_zero, axis=0)
    img_data = np.delete(img_data, y_zero, axis=1)
    
    return img_data

In [ ]:
def cut_3D_image(img_data):
    
    img_data = np.delete(img_data, x_zero, axis=0)
    img_data = np.delete(img_data, y_zero, axis=1)
    img_data = np.delete(img_data, z_zero, axis=2)
    
    x_res = img_data.shape[0] - img_data.shape[0] % x_parts
    y_res = img_data.shape[1] - img_data.shape[1] % y_parts
    z_res = img_data.shape[2] - img_data.shape[2] % z_parts
    img_data = img_data[: x_res, : y_res, : z_res]
    
    return img_data

In [ ]:
def preprocess_2D_image(img_data):
    
    img_data = np.uint16(img_data)
    mask = np.zeros_like(img_data)
    mask = img_data == 0
    
    if use_gaussian:
        img_data = gaussian(img_data, sigma_for_images)
    if use_median:
        img_data = median(img_data, square(3))
    if use_rescale_intensity:
        p2, p98 = np.percentile(img_data, (2, 98))
        img_data = exposure.rescale_intensity(img_data, in_range=(p2, p98))
    if use_equalize_hist:
        img_data = exposure.equalize_hist(img_data)
    if use_adapthist:
        img_data = exposure.equalize_adapthist(img_data, clip_limit=clip_limit)
    
    img_data[mask] = 0
    
    return img_data

#### Make a slice from 3D image

In [ ]:
def slice_image(img_data, axis='z', depth=90, save=False, file='test.nii'):
    
    if axis=='x':
        img_data = img_data[depth,:,:]
    elif axis=='y':
        img_data = img_data[:,depth,:]
    elif axis=='z':
        img_data = img_data[:,:,depth]
    
    if save==True:
        filename = file[0:-4] + "_" + axis + "_" + str(depth) + ".png"
        savepath = os.path.join("images", filename)
        imsave(savepath, img_data)

    return img_data

In [ ]:
def find_segmentation(img_data, file):
    
    global segmentation
    
    img_data = np.uint16(img_data)
    mask = np.zeros_like(img_data)
    mask = img_data == 0
    
    img_data = gaussian(img_data, sigma_for_segmentation)
#     img_data = ndi.gaussian_filter(img_data, sigma)
    img_data = exposure.equalize_hist(img_data)
    img_data[mask] = 0
    
    if use_disk:
        markers = rank.gradient(img_data, disk(1)) < 1
    else:
        markers = rank.gradient(img_data, square(3)) < 1
    
    markers = morphology.remove_small_objects(markers, min_object_size)
    markers = ndi.label(markers)[0]
    elevation_map = sobel(img_data)
    segmentation = watershed(elevation_map, markers, mask=~mask)
    
    plt.figure()
    plt.imshow(img_data, cmap='gray')
    plt.imshow(segmentation, cmap='spectral', alpha=0.4)

    savepath = os.path.join("images", file[0:-4] + "_" + date + ".png")
    plt.savefig(savepath)

#### Calculate 2D histogram for a given image

In [ ]:
def count_2D_segm_histograms(src_path, file):
    
    img_data = load_image(src_path, file)
    img_data = slice_image(img_data, axis=axis, depth=depth)
    img_data = cut_2D_image(img_data)
    img_data = preprocess_2D_image(img_data)
    
    if save_images:
        plt.figure()
        plt.imshow(img_data, cmap='gray')
        plt.imshow(segmentation, cmap='spectral', alpha=0.4)

        savepath = os.path.join("images", file[0:-4] + ".png")
        plt.savefig(savepath)
    
    labels = np.unique(segmentation)
    labels = np.delete(labels, np.where(labels == 0))
    part_array = np.array([])
    
    for i in labels:
        indeces = segmentation == i
        img_part = img_data[indeces]
        hist, bin_edges = np.histogram(img_part, range=(0,1), bins=bins)
        part_array = np.hstack((part_array, hist))
        part_array = np.append(part_array, np.mean(img_part))
        part_array = np.append(part_array, np.var(img_part))
        part_array = np.append(part_array, np.median(img_part))
        
        if save_histograms:
            plt.figure()
            plt.hist(img_part, range=(clip_limit,1-clip_limit), bins=bins)
            savepath = os.path.join("images", "histograms", file[0:-4] + "_hist_" + str(i) + ".png")
            plt.savefig(savepath)
        
    return part_array

In [ ]:
def initial_configuration():
    
    global date, X_filename, X_test_filename, src_train_path, src_test_path, train_names, test_names
    
    today = datetime.datetime.today()
    date_format = "%d%m%y_%H%M%S"
    date = today.strftime(date_format)
    
    src_train_path = os.path.join(os.getcwd(), "..", "data", "set_train")
    src_test_path = os.path.join(os.getcwd(), "..", "data", "set_test")
    
    train_filepaths = os.path.join(src_train_path, "*.nii")
    train_paths = (glob.glob(train_filepaths))
    train_names = [os.path.basename(x) for x in train_paths]
    train_names = natsorted(train_names)

    test_filepaths = os.path.join(src_test_path, "*.nii")
    test_paths = (glob.glob(test_filepaths))
    test_names = [os.path.basename(x) for x in test_paths]
    test_names = natsorted(test_names)

#### Function to calculate histograms for train and test data. The same number of bins is used for all images

In [ ]:
def calculate_2D_histogram_arrays():

    global x_zero, y_zero
    
    file = train_names[image_to_generate_segments]
    img_data = load_image(src_train_path, file)
    img_data = slice_image(img_data, axis=axis, depth=depth)
    
    x_zero = np.where(np.all(img_data==0, axis=1) == 1)
    y_zero = np.where(np.all(img_data==0, axis=0) == 1)
    
    img_data = cut_2D_image(img_data)
    find_segmentation(img_data, file)
    
    for i, file in enumerate(train_names):
        
        part_array = count_2D_segm_histograms(src_train_path, file)
        
        if i == 0:
            X = part_array
        else:
            X = np.vstack((X, part_array))
            
    for i, file in enumerate(test_names):
        
        part_array = count_2D_segm_histograms(src_test_path, file)
        
        if i == 0:
            X_test = part_array
        else:
            X_test = np.vstack((X_test, part_array))
            
    if use_pca_for_segments:
    
        scl = StandardScaler()
        X = scl.fit_transform(X)
        X_test = scl.transform(X_test)

        labels = np.unique(segmentation)
        labels = np.delete(labels, np.where(labels == 0))

        for i, label in enumerate(labels):
            X_temp = X[:,range(i*(bins+3), (i+1)*(bins+3))]
            X_test_temp = X_test[:,range(i*(bins+3), (i+1)*(bins+3))]
            pca = PCA(n_components=n_components_for_2D_segments)
            X_temp = pca.fit_transform(X_temp)
            X_test_temp = pca.transform(X_test_temp)

            if i==0:
                X_new = X_temp
                X_test_new = X_test_temp
            else:
                X_new = np.hstack((X_new, X_temp))
                X_test_new = np.hstack((X_test_new, X_test_temp))

        X = X_new
        X_test = X_test_new
        
    return X.astype(np.float64), X_test.astype(np.float64)

In [ ]:
def preprocess_3D_image(img_data):
    
    img_data = np.uint16(img_data)
    mask = np.zeros_like(img_data)
    mask = img_data == 0
#     img_shape = img_data.shape
    
    if use_gaussian:
        img_data = gaussian(img_data, sigma_for_images, multichannel=False)
    if use_median:
        img_data = median(img_data, square(3))
        
    img_data[mask] = 0
    
    return img_data

In [ ]:
def calculate_3D_histogram_arrays():

    global x_zero, y_zero, z_zero, x_pix, y_pix, z_pix
    
    file = train_names[image_to_generate_segments]
    img_data = load_image(src_train_path, file)
    
    x_zero = np.where(np.all(img_data==0, axis=(1,2)) == 1)
    y_zero = np.where(np.all(img_data==0, axis=(0,2)) == 1)
    z_zero = np.where(np.all(img_data==0, axis=(0,1)) == 1)
    
    img_data = cut_3D_image(img_data)
    (x_dim, y_dim, z_dim) = img_data.shape
    x_pix, y_pix, z_pix = chunks(range(x_dim),x_parts), chunks(range(y_dim),y_parts), chunks(range(z_dim),z_parts)
    
    for i, file in enumerate(train_names):
        
        part_array = count_3D_histograms(src_train_path, file)
        
        if i == 0:
            X = part_array
        else:
            X = np.vstack((X, part_array))
            
    for i, file in enumerate(test_names):
        
        part_array = count_3D_histograms(src_test_path, file)
        
        if i == 0:
            X_test = part_array
        else:
            X_test = np.vstack((X_test, part_array))
            
    if use_pca_for_segments:
    
        scl = StandardScaler()
        X = scl.fit_transform(X)
        X_test = scl.transform(X_test)

        for i in range(x_parts*x_parts*z_parts):
            X_temp = X[:,range(i*(bins+3), (i+1)*(bins+3))]
            X_test_temp = X_test[:,range(i*(bins+3), (i+1)*(bins+3))]
            pca = PCA(n_components=n_components_for_3D_segments)
            X_temp = pca.fit_transform(X_temp)
            X_test_temp = pca.transform(X_test_temp)

            if i==0:
                X_new = X_temp
                X_test_new = X_test_temp
            else:
                X_new = np.hstack((X_new, X_temp))
                X_test_new = np.hstack((X_test_new, X_test_temp))

        X = X_new
        X_test = X_test_new
        
    return X.astype(np.float64), X_test.astype(np.float64)

In [ ]:
def count_3D_histograms(src_path, file):
    
    img_data = load_image(src_path, file)
    img_data = cut_3D_image(img_data)
    img_data = preprocess_3D_image(img_data)
    
    part_array = np.array([])
    
    for j in range(x_parts):
        x_range = x_pix[j]

        for k in range(y_parts):
            y_range = y_pix[k]

            for l in range (z_parts):
                z_range = z_pix[l]

                indeces = np.ix_(x_range, y_range, z_range)
                img_part = img_data[indeces]
                
                hist, bin_edges = np.histogram(img_part, range=(clip_limit,1-clip_limit), bins=bins)

                part_array = np.hstack((part_array, hist))
                part_array = np.append(part_array, np.mean(img_part))
                part_array = np.append(part_array, np.var(img_part))
                part_array = np.append(part_array, np.median(img_part))
        
    return part_array

In [ ]:
def learn(X_train, y_train, X_valid, y_valid, X_test, use_pca=0):

    clfs = []
    
    param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
    param_grid = {
        'clf__C': param_range, 
        'clf__gamma': param_range, 
        'clf__kernel': ['linear', 'rbf'],
        'clf__class_weight' : [None, 'balanced']
    }
    
    if use_pca:
        pipe_svc = Pipeline([('scl', StandardScaler()), ('pca', PCA()), ('clf', SVC(random_state=1))])
        param_grid['pca__n_components'] = [5, 10, 20]
    else:
        pipe_svc = Pipeline([('clf', SVC(random_state=1, probability=True))])
        
    clf_svc = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, cv=cv_inner_folds, n_jobs=-1)
    
    clfs.append(clf_svc.fit(X_train, y_train))
#     scores_svc = cross_val_score(clf_svc, X_train, y_train, cv=cv_outer_folds, n_jobs=1)
    
    param_grid = {
        'rfc__n_estimators': [10, 50, 100, 200],
        'rfc__max_features': [None, 'auto', 'log2'],
        'rfc__class_weight': [None, 'balanced', 'balanced_subsample'],
        'rfc__bootstrap' : [False, True]
    }

    if use_pca:
        pipe_rfc = Pipeline([('scl', StandardScaler()), ('pca', PCA()), ('rfc', RandomForestClassifier(n_jobs=-1))])
        param_grid['pca__n_components'] = [5, 10, 20]
    else:
        pipe_rfc = Pipeline([('rfc', RandomForestClassifier(n_jobs=-1))])

    clf_rfc = GridSearchCV(estimator=pipe_rfc, param_grid=param_grid, cv=cv_inner_folds, n_jobs=-1)

    clfs.append(clf_rfc.fit(X_train, y_train))
#     scores_rfc = cross_val_score(clf_rfc, X_train, y_train, cv=cv_outer_folds, n_jobs=1) 
    
    param_grid = {
        'bc__n_estimators': [10, 50, 100, 200],
        'bc__max_features': [5, 10, 20],
        'bc__bootstrap' : [True, False],
        'bc__bootstrap_features' : [True, False]
    }

    if use_pca:
        pipe_bc = Pipeline([('scl', StandardScaler()), ('pca', PCA()), ('bc', BaggingClassifier())])
        param_grid['pca__n_components'] = [5, 10, 20]
    else:
        pipe_bc = Pipeline([('bc', BaggingClassifier())])

    clf_bc = GridSearchCV(estimator=pipe_bc, param_grid=param_grid, cv=cv_inner_folds, n_jobs=-1)

    clfs.append(clf_bc.fit(X_train, y_train))

    return clfs

In [ ]:
def learn_multilabel(X_train, y_train, X_valid, y_valid, X_test, use_pca=0):

    clfs = []
    
    
    param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
    param_grid = {
        'clf__estimator__C': param_range, 
        'clf__estimator__gamma': param_range, 
        'clf__estimator__kernel': ['linear', 'rbf'],
        'clf__estimator__class_weight' : [None, 'balanced']
    }
    
    if use_pca:
        pipe_svc = Pipeline([('scl', StandardScaler()), ('pca', PCA()), ('clf', OneVsOneClassifier(SVC(random_state=1, probability=True)))])
        param_grid['pca__n_components'] = [5, 10, 20]
    else:
        pipe_svc = Pipeline([('clf', OneVsRestClassifier(SVC(random_state=1, probability=True)))])
        
    clf_svc = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, cv=cv_inner_folds, n_jobs=-1)
    
    clfs.append(clf_svc.fit(X_train, y_train))
    
    
    param_grid = {
        'rfc__n_estimators': [10, 50, 100, 200],
        'rfc__max_features': [None, 'auto', 'log2'],
        'rfc__class_weight': [None, 'balanced', 'balanced_subsample'],
        'rfc__bootstrap' : [False, True]
    }

    if use_pca:
        pipe_rfc = Pipeline([('scl', StandardScaler()), ('pca', PCA()), ('rfc', RandomForestClassifier(n_jobs=-1))])
        param_grid['pca__n_components'] = [5, 10, 20]
    else:
        pipe_rfc = Pipeline([('rfc', RandomForestClassifier(n_jobs=-1))])

    clf_rfc = GridSearchCV(estimator=pipe_rfc, param_grid=param_grid, cv=cv_inner_folds, n_jobs=-1)

    clfs.append(clf_rfc.fit(X_train, y_train))
    
    return clfs

#### Calculating X (all training samples derived from given training images), X_test (test samples derived from given test images) and y (targets for given training images) matrices

In [ ]:
initial_configuration()

y = pd.read_csv("targets.csv", header=None).values

#####################################################

axis = 'y'
depth = 100
image_to_generate_segments = 112
sigma_for_segmentation = 1.5
use_disk = True
min_object_size = 2

X, X_test = calculate_2D_histogram_arrays()

#####################################################

axis = 'x'
depth = 90
image_to_generate_segments = 162
sigma_for_segmentation = 1.2
use_disk = True
min_object_size = 4

X_temp, X_test_temp = calculate_2D_histogram_arrays()
X = np.hstack((X, X_temp))
X_test = np.hstack((X_test, X_test_temp))

#####################################################

axis = 'z'
depth = 92
image_to_generate_segments = 150
sigma_for_segmentation = 4
use_disk = True
min_object_size = 2

X_temp, X_test_temp = calculate_2D_histogram_arrays()
X = np.hstack((X, X_temp))
X_test = np.hstack((X_test, X_test_temp))

#####################################################

if use_3D:
    X_temp, X_test_temp = calculate_3D_histogram_arrays()
    X = np.hstack((X, X_temp))
    X_test = np.hstack((X_test, X_test_temp))

#####################################################

if use_pca_before_learning:
    scl = StandardScaler()
    X = scl.fit_transform(X)
    X_test = scl.transform(X_test)
    pca = PCA(n_components=n_components_before_learning)
    X = pca.fit_transform(X)
    X_test = pca.transform(X_test)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=2)

#### Predict age

In [ ]:
y_train_age = y_train[:,1:2]
y_valid_age = y_valid[:,1:2]

clfs_age = learn(X_train, y_train_age.ravel(), X_valid, y_valid_age.ravel(), X_test, use_pca=use_pca_for_learning)

In [ ]:
y_valid_pred_age = np.zeros((X_valid.shape[0], len(clfs_age)))
y_test_pred_age = np.zeros((X_test.shape[0], len(clfs_age)))

for i, clf in enumerate(clfs_age):

    y_train_pred_age = clf.predict(X_train)
    y_valid_pred_age_ = clf.predict(X_valid)

    y_valid_pred_age[:,i] = clf.predict_proba(X_valid)[:,1]
    y_test_pred_age[:,i] = clf.predict_proba(X_test)[:,1]

    train_error = hamming_loss(y_train_age, y_train_pred_age)
    valid_error = hamming_loss(y_valid_age, y_valid_pred_age_)

    train_confusion_array = confusion_matrix(y_train_age, y_train_pred_age)
    valid_confusion_array = confusion_matrix(y_valid_age, y_valid_pred_age_)

    print(clf.best_score_)
    print(clf.best_params_)

    print(train_error)
    print(valid_error)

    print(train_confusion_array)
    print(valid_confusion_array)

y_valid_pred_age = np.round(np.mean(y_valid_pred_age, axis=1)).astype(int)
y_valid_pred_age = np.reshape(y_valid_pred_age, (y_valid_pred_age.size,1))

y_test_pred_age = np.round(np.mean(y_test_pred_age, axis=1)).astype(int)
y_test_pred_age = np.reshape(y_test_pred_age, (y_test_pred_age.size,1))

valid_error = hamming_loss(y_valid_age, y_valid_pred_age)
valid_confusion_array = confusion_matrix(y_valid_age, y_valid_pred_age)

print(valid_error)
print(valid_confusion_array)

In [ ]:
y_train_sick = y_train[:,2:3]
y_valid_sick = y_valid[:,2:3]
y_train_sex = y_train[:,0:1]
y_valid_sex = y_valid[:,0:1]

X_train_new = np.hstack((X_train, y_train_age))
X_valid_new = np.hstack((X_valid, y_valid_pred_age))
X_test_new = np.hstack((X_test, y_test_pred_age))

clfs = learn_multilabel(X_train_new, y_train, X_valid_new, y_valid, X_test_new, use_pca=use_pca_for_learning)

In [ ]:
y_valid_pred = np.zeros((X_valid.shape[0], 3, len(clfs)))
y_test_pred = np.zeros((X_test.shape[0], 3, len(clfs)))

for i, clf in enumerate(clfs):

    y_train_pred = clf.predict(X_train_new)
    y_valid_pred_ = clf.predict(X_valid_new)
    
    train_error = hamming_loss(y_train, y_train_pred)
    valid_error = hamming_loss(y_valid, y_valid_pred_)

    print(clf.best_score_)
    print(clf.best_params_)

    print(train_error)
    print(valid_error)

for j in range(3):
    y_valid_pred[:,j,0] = clfs[0].predict_proba(X_valid_new)[:,j]
    y_test_pred[:,j,0] = clfs[0].predict_proba(X_test_new)[:,j]
    
    y_valid_pred[:,j,1] = clfs[1].predict_proba(X_valid_new)[j][:,1]
    y_test_pred[:,j,1] = clfs[1].predict_proba(X_test_new)[j][:,1]
    
y_valid_pred = np.round(np.mean(y_valid_pred, axis=2)).astype(int)
y_test_pred = np.round(np.mean(y_test_pred, axis=2)).astype(int)

print(y_test_pred)

valid_error = hamming_loss(y_valid, y_valid_pred)

print(valid_error)

#### Pipeline with standarizing data and classifying them

In [ ]:
valid_error = hamming_loss(y_valid, y_valid_pred)

print(valid_error)

parameters = {
    'axis' : axis,
    'depth' : depth,
    'clip_limit' : clip_limit,
    'bins' : bins,
    'image_to_generate_segments' : image_to_generate_segments,

    'use_gaussian' : use_gaussian,
    'use_median' : use_median,
    
    'sigma_for_segmentation' : sigma_for_segmentation,
    'sigma_for_images' : sigma_for_images,

    'use_equalize_hist' : use_equalize_hist,
    'use_adapthist' : use_adapthist,
    'use_rescale_intensity' : use_rescale_intensity,

    'save_images' : save_images,
    'save_histograms' : save_histograms,

    'use_pca_for_segments' : use_pca_for_segments,
    'use_pca_for_learning' : use_pca_for_learning,
    'use_pca_before_learning' : use_pca_before_learning,
    
    'n_components_for_2D_segments' : n_components_for_2D_segments,
    'n_components_for_3D_segments' : n_components_for_3D_segments,
    'n_components_before_learning' : n_components_before_learning,
    
    'cv_inner_folds' : cv_inner_folds
}

savepath = os.path.join("results", "output_" + date + ".txt")
with open(savepath, "w") as text_file:
    text_file.write("Date: " + str(date) + "\n")
    text_file.write("Valid error: " + str(valid_error) + "\n")
    text_file.write("Parameters: " + str(parameters) + "\n")

#### Saving scores obtained from original test images to produce file for submission

In [ ]:
y_test_pred = y_test_pred.ravel()

nr = np.arange(0,414)
label = np.array(138*["gender", "age", "health"])
sample = np.arange(0,138)
sample = np.dstack((sample, sample, sample))
sample = sample.ravel()
label = label.ravel()

savepath = os.path.join("results", "submission_" + date + ".csv")
data = {"ID" : nr, "Sample" : sample, "Label" : label, "Predicted" : y_test_pred}
df = pd.DataFrame(data, columns=['ID', 'Sample', 'Label', 'Predicted'])
df.to_csv(savepath, index=False)